In [44]:
# IMPORT Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from tensorflow.keras.utils import to_categorical
import numpy as np

In [45]:
# Import the Australian Road Death Dataset
ardd_data = pd.read_csv('../Resources/ardd_fatal_crashes.csv')
ardd_data

/var/folders/h4/v2g57xvj6mz75vxd96qdxsqw0000gn/T/ipykernel_2989/2073364016.py:2: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ardd_data = pd.read_csv('../Resources/ardd_fatal_crashes.csv')


,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day
0,20221049,NSW,3,2022,Tuesday,06:50,Single,1,No,Yes,No,90,Inner Regional Australia,Capital Region,Eurobodalla,Arterial Road,No,No,Weekday,Day
1,20222018,Vic,3,2022,Friday,13:05,Multiple,1,-9,-9,-9,-9,Major Cities of Australia,Melbourne - Inner,Port Phillip (C),Local Road,No,No,Weekday,Day
2,20224006,SA,3,2022,Friday,09:30,Multiple,1,No,No,No,80,Outer Regional Australia,South Australia - Outback,Port Augusta (C),National or State Highway,No,No,Weekday,Day
3,20225008,WA,3,2022,Saturday,06:00,Single,1,No,No,No,110,NaN,NaN,NaN,NaN,No,No,Weekend,Day
4,20226010,Tas,3,2022,Tuesday,11:47,Single,1,No,No,No,100,Outer Regional Australia,Launceston and North East,Break O'Day (M),National or State Highway,No,No,Weekday,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48116,19892204,Vic,1,1989,Friday,13:20,Single,1,No,No,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48117,19893326,Qld,1,1989,Monday,09:00,Multiple,1,No,-9,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48118,19892576,Vic,1,1989,Friday,17:15,Multiple,1,No,No,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day
48119,19891122,NSW,1,1989,Tuesday,14:10,Multiple,1,No,-9,No,60,NaN,NaN,NaN,NaN,No,No,Weekday,Day


In [46]:
#check the shape of the original CSV
ardd_data.shape

(48121, 20)

In [47]:
cleaned_ardd_data1 = ardd_data.filter(['State','Month','Year','Crash Type','Speed Limit','National Road Type','National Remoteness Areas', 'Christmas Period','Easter Period','Day of week','Time of Day'])
cleaned_ardd_data1.head(20)

,State,Month,Year,Crash Type,Speed Limit,National Road Type,National Remoteness Areas,Christmas Period,Easter Period,Day of week,Time of Day
0,NSW,3,2022,Single,90,Arterial Road,Inner Regional Australia,No,No,Weekday,Day
1,Vic,3,2022,Multiple,-9,Local Road,Major Cities of Australia,No,No,Weekday,Day
2,SA,3,2022,Multiple,80,National or State Highway,Outer Regional Australia,No,No,Weekday,Day
3,WA,3,2022,Single,110,NaN,NaN,No,No,Weekend,Day
4,Tas,3,2022,Single,100,National or State Highway,Outer Regional Australia,No,No,Weekday,Day
5,NSW,3,2022,Multiple,70,National or State Highway,Major Cities of Australia,No,No,Weekend,Day
6,NSW,3,2022,Single,60,Arterial Road,Major Cities of Australia,No,No,Weekend,Day
7,Vic,3,2022,Multiple,60,Sub-arterial Road,Inner Regional Australia,No,No,Weekend,Day
8,Vic,3,2022,Multiple,100,Arterial Road,Inner Regional Australia,No,No,Weekend,Day
9,Qld,3,2022,Single,100,Local Road,Inner Regional Australia,No,No,Weekday,Day


In [48]:
# according to the documentation, -9 would mean "Unknown", however after data exploration those data will be of no use for our ML model
cleaned_ardd_data1 = cleaned_ardd_data1.replace("-9", np.nan)

In [49]:
cleaned_ardd_data1 = cleaned_ardd_data1.replace("<40", 40)

In [50]:
cleaned_ardd_data1 = cleaned_ardd_data1.replace(100, 110)

In [51]:
cleaned_ardd_data1 = cleaned_ardd_data1.replace(60, 70)

In [52]:
cleaned_ardd_data1 = cleaned_ardd_data1.replace(70, 80)

In [53]:
cleaned_ardd_data1 = cleaned_ardd_data1.replace("Unspecified", np.nan)

In [54]:
#drop rows containing null values
cleaned_ardd_data1 = cleaned_ardd_data1.dropna()

In [55]:
#check shape 
cleaned_ardd_data1.shape

(7126, 11)

In [56]:
cleaned_ardd_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7126 entries, 0 to 8954
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   State                      7126 non-null   object
 1   Month                      7126 non-null   int64 
 2   Year                       7126 non-null   int64 
 3   Crash Type                 7126 non-null   object
 4   Speed Limit                7126 non-null   object
 5   National Road Type         7126 non-null   object
 6   National Remoteness Areas  7126 non-null   object
 7   Christmas Period           7126 non-null   object
 8   Easter Period              7126 non-null   object
 9   Day of week                7126 non-null   object
 10  Time of Day                7126 non-null   object
dtypes: int64(2), object(9)
memory usage: 668.1+ KB


In [57]:
cleaned_ardd_data1

,State,Month,Year,Crash Type,Speed Limit,National Road Type,National Remoteness Areas,Christmas Period,Easter Period,Day of week,Time of Day
0,NSW,3,2022,Single,90,Arterial Road,Inner Regional Australia,No,No,Weekday,Day
2,SA,3,2022,Multiple,80,National or State Highway,Outer Regional Australia,No,No,Weekday,Day
4,Tas,3,2022,Single,100,National or State Highway,Outer Regional Australia,No,No,Weekday,Day
5,NSW,3,2022,Multiple,70,National or State Highway,Major Cities of Australia,No,No,Weekend,Day
6,NSW,3,2022,Single,60,Arterial Road,Major Cities of Australia,No,No,Weekend,Day
...,...,...,...,...,...,...,...,...,...,...,...
8936,Tas,1,2014,Multiple,80,National or State Highway,Inner Regional Australia,No,No,Weekday,Day
8944,WA,1,2014,Single,100,NATIONAL OR STATE HIGHWAY,Major Cities of Australia,Yes,No,Weekday,Day
8948,SA,1,2014,Single,50,Local Road,Major Cities of Australia,No,No,Weekday,Night
8950,SA,1,2014,Single,50,Local Road,Major Cities of Australia,No,No,Weekday,Day


In [58]:
cleaned_ardd_data = cleaned_ardd_data1.rename(columns={'Crash Type': 'crash_type','Bus \nInvolvement': 'bus_involvement','Heavy Rigid Truck Involvement': 'heavy_rigid_truck_involvement','Articulated Truck Involvement': 'articulated_truck_involvement','Speed Limit': 'speed_limit','Christmas Period': 'christmas_period','Easter Period': 'easter_period','Day of week': 'day_of_week'})

In [59]:
# Set features. 
X = cleaned_ardd_data.drop("crash_type", axis=1)
y = cleaned_ardd_data["crash_type"]
print(X.shape, y.shape)

(7126, 10) (7126,)


In [60]:
X["speed_limit"].unique()

array(['90', '80', '100', '70', '60', '110', '50', '130', '40', '5', '10',
       '30', '25', '20', 40, '75'], dtype=object)

In [61]:
X["Year"].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014])

In [62]:
y.unique()

array(['Single', 'Multiple'], dtype=object)

In [63]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

transformed_X = onehotencoder.fit_transform(X[["State","christmas_period", "National Road Type","National Remoteness Areas","easter_period","day_of_week","Time of Day"]])
print(transformed_X)

  (0, 1)	1.0
  (0, 8)	1.0
  (0, 13)	1.0
  (0, 23)	1.0
  (0, 28)	1.0
  (0, 30)	1.0
  (0, 32)	1.0
  (1, 4)	1.0
  (1, 8)	1.0
  (1, 19)	1.0
  (1, 25)	1.0
  (1, 28)	1.0
  (1, 30)	1.0
  (1, 32)	1.0
  (2, 5)	1.0
  (2, 8)	1.0
  (2, 19)	1.0
  (2, 25)	1.0
  (2, 28)	1.0
  (2, 30)	1.0
  (2, 32)	1.0
  (3, 1)	1.0
  (3, 8)	1.0
  (3, 19)	1.0
  (3, 24)	1.0
  :	:
  (7122, 24)	1.0
  (7122, 28)	1.0
  (7122, 30)	1.0
  (7122, 32)	1.0
  (7123, 4)	1.0
  (7123, 8)	1.0
  (7123, 17)	1.0
  (7123, 24)	1.0
  (7123, 28)	1.0
  (7123, 30)	1.0
  (7123, 33)	1.0
  (7124, 4)	1.0
  (7124, 8)	1.0
  (7124, 17)	1.0
  (7124, 24)	1.0
  (7124, 28)	1.0
  (7124, 30)	1.0
  (7124, 32)	1.0
  (7125, 7)	1.0
  (7125, 8)	1.0
  (7125, 11)	1.0
  (7125, 24)	1.0
  (7125, 28)	1.0
  (7125, 31)	1.0
  (7125, 33)	1.0


In [64]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
numeric_features = ["Month", "Year","speed_limit"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["State","christmas_period", "National Road Type","National Remoteness Areas","easter_period","day_of_week","Time of Day"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.627


/Users/andrianichristanty/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [66]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    transformed_X, encoded_y, random_state=1)


In [69]:
X_train

<5344x34 sparse matrix of type '<class 'numpy.float64'>'
	with 37408 stored elements in Compressed Sparse Row format>

In [70]:
y_train

array([0, 1, 1, ..., 0, 1, 1])

In [71]:
y_test

array([1, 0, 1, ..., 0, 1, 1])

In [72]:
X_test

<1782x34 sparse matrix of type '<class 'numpy.float64'>'
	with 12474 stored elements in Compressed Sparse Row format>

In [73]:
from sklearn import tree

In [74]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_train, y_train)

0.686564371257485

In [75]:
clf = clf.fit(X_train, y_train)
clf.score(X_train, y_train)

0.686564371257485

In [78]:
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")

Training Data Score: 0.686564371257485
Testing Data Score: 0.6060606060606061
